In [ ]:
import pandas as pd
import requests
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo

In [ ]:
headers = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [ ]:
from nba_api.stats.endpoints import commonallplayers

def get_active_players_for_season(season='2020-21'):
    # Fetch all players from the NBA API
    all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1, league_id='00', season=season)
    
    # Convert the fetched data to a DataFrame
    players_data = all_players.common_all_players.get_data_frame()
    
    # Filter active players
    active_players = players_data[players_data['ROSTERSTATUS'] == 1]
    
    return active_players[['PERSON_ID', 'DISPLAY_FIRST_LAST', 'TEAM_NAME', 'TEAM_CITY']]

# Example usage
nba_players = get_active_players_for_season()
df = pd.DataFrame(nba_players)
df.sample(5)
active_player_ids = df.loc[(df['ROSTERSTATUS'] == 1), 'PERSON_ID'].to_list()
active_player_names = df.loc[(df['ROSTERSTATUS'] == 1), 'DISPLAY_FIRST_LAST'].to_list()
active_player_ids_test = active_player_ids[10:40]
active_player_names_test = active_player_names[10:40]
active_player_names_test

In [ ]:
def get_matchups(id, max_retries=3):
    import requests
    import time

    url = "https://stats.nba.com/stats/leagueseasonmatchups"

    params = {
        "DateFrom": "",
        "DateTo": "",
        "DefPlayerID": id,
        "LeagueID": "00",
        "Outcome": "",
        "PORound": "0",
        "PerMode": "Totals",
        "Season": "2020-21",
        "SeasonType": "Regular Season"
    }

    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip, deflate",
        "Referer": "https://www.nba.com/",
        "User-Agent": "Mozilla/5.0",
        "x-nba-stats-origin": "stats",
        "x-nba-stats-token": "true"
    }

    attempt = 0
    while attempt < max_retries:
        try:
            print(f"Attempting to fetch matchups for player ID# {id} (Attempt {attempt + 1}/{max_retries})...")
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()

            fieldnames = data["resultSets"][0]["headers"]
            for row in data["resultSets"][0]["rowSet"]:
                yield dict(zip(fieldnames, row))
            return  # Exit the loop if successful

        except requests.exceptions.HTTPError as e:
            print(f"HTTPError for player ID# {id}: {e}. Retrying...")
            attempt += 1
            time.sleep(2)  # Delay between retries

        except Exception as e:
            print(f"Unexpected error for player ID# {id}: {e}")
            return  # Stop retries for unexpected errors

    print(f"Failed to fetch data for player ID# {id} after {max_retries} attempts.")
    return


def main():
    from csv import DictWriter

    for id,name in zip(active_player_ids,active_player_names):#Change this line to testing data for testing samples
        all_matchups = list(get_matchups(id))

        if not all_matchups:
            print(f"No valid matchups for player ID# {id}, name: {name}. Skipping...")
            continue

        print(f"Writing to CSV file for player ID# {id} name {name}...")

        with open(f'{name}.csv', "w", newline="") as file:
            fieldnames = list(all_matchups[0].keys())  # Extract headers dynamically
            writer = DictWriter(file, fieldnames=fieldnames)

            writer.writeheader()
            for matchup in all_matchups:
                writer.writerow(matchup)

        print(f"Done writing for player ID# {id}.")

    return 0



if __name__ == "__main__":
    import sys
    sys.exit(main())

In [ ]:
def fetch_player_team_ids(player_ids):
    """
    Fetches the team IDs for a list of player IDs.
    :param player_ids: List of player IDs.
    :return: Dictionary of player IDs and their corresponding
    team IDs.
    """
    team_ids = {}
    for player_id in player_ids:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        info = player_info.common_player_info.get_dict()
        data = info['data']
        if data:
            team_id = data[0][info['headers'].index('TEAM_ID')]
            team_ids[player_id] = team_id
        else:
            team_ids[player_id] = None  # Handling cases where the player may not have a current team
    return team_ids


In [ ]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo

def find_team_id_by_player_name(player_name):
    # Retrieve all NBA players
    nba_players = players.get_players()
    
    # Find player by name (case insensitive match)
    player = [p for p in nba_players if p['full_name'].lower() == player_name.lower()]
    
    if not player:
        return "No player found with that name."

    # Use the first matched player's ID to fetch detailed player information
    player_id = player[0]['id']
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_normalized_dict()

    # Extract team information
    if 'CommonPlayerInfo' in player_info and player_info['CommonPlayerInfo']:
        team_id = player_info['CommonPlayerInfo'][0]['TEAM_ID']
        team_name = player_info['CommonPlayerInfo'][0]['TEAM_NAME']
        return team_id, team_name
    else:
        return "No current team found for this player."

# Test the function
player_name = "LeBron James"
team_id, team_name = find_team_id_by_player_name(player_name)
print(f"{player_name} plays for {team_name} (Team ID: {team_id}).")


## Load dictionary player_data

In [ ]:
from nba_api.stats.endpoints import PlayerCareerStats
from nba_api.stats.static import players, teams

def get_player_team_info(player_name, season='2020-21'):
    # Find the player's information
    player_dict = players.find_players_by_full_name(player_name)
    if not player_dict:
        return f"Player '{player_name}' not found."
    player_info = player_dict[0]
    player_id = player_info['id']

    # Retrieve the player's career stats
    career_stats = PlayerCareerStats(player_id=player_id)
    regular_season_stats = career_stats.get_data_frames()[0]

    # Filter stats for the specified season
    season_stats = regular_season_stats[regular_season_stats['SEASON_ID'] == season]
    if season_stats.empty:
        return f"No data found for {player_name} in the {season} season."

    # Get the team ID
    team_id = season_stats.iloc[0]['TEAM_ID']

    # Find the team name using the team ID
    team_info = teams.find_team_name_by_id(team_id)
    team_name = team_info['full_name'] if team_info else 'Unknown Team'

    return player_id, team_id, team_name

# Example usage:
player_name = "Jarrett Culver"
player_id, team_id, team_name = get_player_team_info(player_name)
print(f"{player_name} (ID: {player_id}) plays for {team_name} (Team ID: {team_id}).")


In [ ]:
import os
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playercareerstats

# Retrieve all NBA players once to reduce API calls
all_nba_players = players.get_players()

def find_team_details_by_name_and_season(player_name, season="2020-21"):
    # Find player by name (case insensitive match)
    player = next((p for p in all_nba_players if p['full_name'].lower() == player_name.lower()), None)
    
    if not player:
        return None, None, "No player found with that name."

    # Fetch the player's career stats
    player_id = player['id']
    career_stats = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]

    # Filter the stats for the specified season
    season_stats = career_stats[career_stats['SEASON_ID'] == season]

    if season_stats.empty:
        return player_id, None, f"No data found for {player_name} in the {season} season."

    # Extract team information
    team_id = season_stats.iloc[0]['TEAM_ID']
    team_abbreviation = season_stats.iloc[0]['TEAM_ABBREVIATION']
    return player_id, team_id, team_abbreviation



player_data = {}
folder_path = r"Desktop/desktop/python/Intro_systemModelling/Final_Project/" 
#Read Your own downloaded Folder_path for file "player_data.pkl"
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        try:
            # Read CSV file
            df = pd.read_csv(file_path)
            
            # Assuming 'MATCHUP_TIME_SEC' is the name of the last column
            last_column = df.columns[-1]

            # Filter DataFrame
            df = df[df[last_column] >= 120]
            
            player_name = os.path.splitext(file_name)[0]
            
            # Find player ID and team details
            player_id, team_id, team_name = get_player_team_info(player_name, season='2020-21')
            if player_id:
                player_data[player_name] = {'player_id': player_id, 'team_id': team_id, 'team_name': team_name, 'data_frame': df}
            print(f"Successfully read and updated: {file_name}")
            
        except Exception as e:
            print(f"Error reading {file_name}: {e}")


In [2]:
# Display the team name for each player
for player_name, data in player_data.items():
    team_name = data['team_name']
    print(f"{player_name} plays for {team_name}.")

# save the player_data to pkl
import pickle
with open('player_data.pkl', 'rb') as pickle_file:
    player_data = pickle.load(pickle_file)
    

In [ ]:
# Write a function to calculate the average rows of data_frame in player_data
def calculate_average_rows(player_data):
    """
    Calculate the average number of rows in the 'data_frame' for each player in the player_data dictionary.

    Parameters:
        player_data (dict): A dictionary containing player names as keys and a dictionary of player information as values.

    Returns:
        float: The average number of rows across all player data frames.
    """
    total_rows = 0
    total_players = 0

    for player_name, player_info in player_data.items():
        if 'data_frame' in player_info:
            total_rows += len(player_info['data_frame'])
            print(f"Player: {player_name}, Rows: {len(player_info['data_frame'])}")
            total_players += 1

    if total_players > 0:
        return total_rows / total_players, total_players, total_rows
    else:
        return 0
    
# Test the function
average_rows, total_players, total_rows = calculate_average_rows(player_data)
print(f"Average rows per player: {average_rows:.2f}")
print(f"Total players processed: {total_players}")
print(f"Total rows processed: {total_rows}")



In [ ]:
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo

# Initialize variables to store global min/max values
global_min_time = float('inf')
global_max_time = float('-inf')
global_min_fg_pct = float('inf')
global_max_fg_pct = float('-inf')

# Iterate over each player's data in the dictionary
for player_name, data in player_data.items():
    # print(f"Processing data for {player_name}...")
    df = data['data_frame']  # Access the DataFrame
    
    # Calculate min and max for 'MATCHUP_TIME_SEC'
    min_time_sec = df['MATCHUP_TIME_SEC'].min()
    max_time_sec = df['MATCHUP_TIME_SEC'].max()

    # Update global min/max
    global_min_time = min(global_min_time, min_time_sec)
    global_max_time = max(global_max_time, max_time_sec)

    # Calculate min and max for 'MATCHUP_FG_PCT'
    min_fg_pct = df['MATCHUP_FG_PCT'].min()
    max_fg_pct = df['MATCHUP_FG_PCT'].max()
    
    # Update global min/max
    global_min_fg_pct = min(global_min_fg_pct, min_fg_pct)
    global_max_fg_pct = max(global_max_fg_pct, max_fg_pct)
        
# Output the results
print(f"Global minimum MATCHUP_TIME_SEC: {global_min_time}")
print(f"Global maximum MATCHUP_TIME_SEC: {global_max_time}")
print(f"Global minimum MATCHUP_FG_PCT: {global_min_fg_pct}")
print(f"Global maximum MATCHUP_FG_PCT: {global_max_fg_pct}")

# Iterate over each player's data in the dictionary
for player_name, data in player_data.items():
    df = data['data_frame']  # Access the DataFrame

    # Ensure 'MATCHUP_TIME_SEC' and other columns exist before attempting calculations
    if 'MATCHUP_TIME_SEC' in df.columns and all(col in df.columns for col in ['MATCHUP_BLK', 'PLAYER_PTS', 'MATCHUP_TOV']):
        # Check to avoid division by zero
        df['MATCHUP_TIME_SEC'] = df['MATCHUP_TIME_SEC'].replace(0, pd.NA)  # Replace 0 with NA to avoid division by zero
        
        # Calculate new columns
        df['UNIT_MATCHUP_PTS'] = df['MATCHUP_BLK'] / df['MATCHUP_TIME_SEC'] * 60 * 36
        df['UNIT_MATCHUP_PTS'] = df['PLAYER_PTS'] / df['MATCHUP_TIME_SEC'] * 60 * 36
        df['UNIT_MATCHUP_TOV'] = df['MATCHUP_TOV'] / df['MATCHUP_TIME_SEC'] * 60 * 36
        
        # Update the dictionary with the modified DataFrame
        player_data[player_name]['data_frame'] = df
        print(f"Updated DataFrame for {player_name} with new unit metrics.")

In [ ]:
# Print the data_frame at Aaron Gordon UNIT_MATCHUP_PTS on the row of OFF_PLAYER_NAME is Kawhi Leonard
print(player_data['Aaron Gordon']['data_frame'].loc[player_data['Aaron Gordon']['data_frame']['OFF_PLAYER_NAME'] == 'Kawhi Leonard', 'UNIT_MATCHUP_PTS'])

print(player_data['Aaron Gordon']['team_name'])
player_data['Aaron Gordon']['data_frame']

## UNIT_MATCHUP_BLK with all connections

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Create a graph
G = nx.DiGraph()

# Initialize node color map
node_colors = {}

# Filter players from Lakers and add nodes
lakers_players = {name: data for name, data in player_data.items() if data['team_name'] == 'Los Angeles Lakers'}
for name in lakers_players:
    G.add_node(name)
    node_colors[name] = 'gold'  # Color for Lakers players

# Add other NBA players as nodes
for name in player_data:
    if name not in lakers_players:
        G.add_node(name)
        node_colors[name] = 'gray'  # Color for other players

# Add edges based on UNIT_MATCHUP_TOV values
for name, data in player_data.items():
    df = data['data_frame']
    if 'OFF_PLAYER_NAME' in df.columns and 'UNIT_MATCHUP_TOV' in df.columns:
        for index, row in df.iterrows():
            if row['UNIT_MATCHUP_TOV'] != 0:
                if row['OFF_PLAYER_NAME'] in G.nodes and name in G.nodes:
                    G.add_edge(row['OFF_PLAYER_NAME'], name, weight=row['UNIT_MATCHUP_TOV'])

# Remove nodes with no edges
nodes_with_no_edges = [node for node in G.nodes if G.degree(node) == 0]
G.remove_nodes_from(nodes_with_no_edges)

# Create custom layout
def custom_layout(G, lakers_players, center_radius=0.3, outer_radius=1.0):
    pos = {}
    
    # Position Lakers players in inner circle
    lakers_nodes = [node for node in G.nodes if node in lakers_players]
    n_lakers = len(lakers_nodes)
    for i, node in enumerate(lakers_nodes):
        angle = 2 * np.pi * i / n_lakers
        pos[node] = np.array([
            center_radius * np.cos(angle),
            center_radius * np.sin(angle)
        ])
    
    # Position other players in outer circle
    other_nodes = [node for node in G.nodes if node not in lakers_players]
    n_others = len(other_nodes)
    for i, node in enumerate(other_nodes):
        angle = 2 * np.pi * i / n_others
        pos[node] = np.array([
            outer_radius * np.cos(angle),
            outer_radius * np.sin(angle)
        ])
    
    return pos

# Visualization
plt.figure(figsize=(40, 40))
pos = custom_layout(G, lakers_players)
edges = G.edges(data=True)
weights = [edge[2]['weight'] for edge in edges]

# Draw the network
nx.draw(G, pos,
        node_color=[node_colors[node] for node in G.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('NBA Players UNIT_MATCHUP_TOV Network\nLakers Players (Gold) in Center', fontsize=16)
plt.show()

In [ ]:
# First plot: Block capability
plt.figure(figsize=(40, 40))

# Create subgraph with only edges to Lakers
edges_to_lakers = [(u, v) for (u, v, d) in G.edges(data=True) 
                   if v in lakers_players]
G_blocks = G.edge_subgraph(edges_to_lakers).copy()

# Remove isolated nodes
G_blocks.remove_nodes_from(list(nx.isolates(G_blocks)))

pos = custom_layout(G_blocks, lakers_players)
weights_to_lakers = [G[u][v]['weight'] for u, v in G_blocks.edges()]
node_colors_filtered = {node: node_colors[node] for node in G_blocks.nodes()}

nx.draw(G_blocks, pos,
        node_color=[node_colors_filtered[node] for node in G_blocks.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights_to_lakers,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('Blocks Against Lakers Players', fontsize=16)
plt.show()

# Second plot: Anti-block capability
plt.figure(figsize=(40, 40))

# Create subgraph with only edges from Lakers
edges_from_lakers = [(u, v) for (u, v, d) in G.edges(data=True) 
                     if u in lakers_players]
G_antiblocks = G.edge_subgraph(edges_from_lakers).copy()

# Remove isolated nodes
G_antiblocks.remove_nodes_from(list(nx.isolates(G_antiblocks)))

pos = custom_layout(G_antiblocks, lakers_players)
weights_from_lakers = [G[u][v]['weight'] for u, v in G_antiblocks.edges()]
node_colors_filtered = {node: node_colors[node] for node in G_antiblocks.nodes()}

nx.draw(G_antiblocks, pos,
        node_color=[node_colors_filtered[node] for node in G_antiblocks.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights_from_lakers,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('Blocks By Lakers Players', fontsize=16)
plt.show()

## UNIT_MATCHUP_TOV with all connections

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Create a graph
G = nx.DiGraph()

# Initialize node color map
node_colors = {}

# Filter players from Lakers and add nodes
lakers_players = {name: data for name, data in player_data.items() if data['team_name'] == 'Los Angeles Lakers'}
for name in lakers_players:
    G.add_node(name)
    node_colors[name] = 'gold'  # Color for Lakers players

# Add other NBA players as nodes
for name in player_data:
    if name not in lakers_players:
        G.add_node(name)
        node_colors[name] = 'gray'  # Color for other players

# Add edges based on UNIT_MATCHUP_TOV values
for name, data in player_data.items():
    df = data['data_frame']
    if 'OFF_PLAYER_NAME' in df.columns and 'UNIT_MATCHUP_TOV' in df.columns:
        for index, row in df.iterrows():
            if row['UNIT_MATCHUP_TOV'] != 0:
                if row['OFF_PLAYER_NAME'] in G.nodes and name in G.nodes:
                    G.add_edge(row['OFF_PLAYER_NAME'], name, weight=row['UNIT_MATCHUP_TOV'])

# Remove nodes with no edges
nodes_with_no_edges = [node for node in G.nodes if G.degree(node) == 0]
G.remove_nodes_from(nodes_with_no_edges)

# Create custom layout
def custom_layout(G, lakers_players, center_radius=0.3, outer_radius=1.0):
    pos = {}
    
    # Position Lakers players in inner circle
    lakers_nodes = [node for node in G.nodes if node in lakers_players]
    n_lakers = len(lakers_nodes)
    for i, node in enumerate(lakers_nodes):
        angle = 2 * np.pi * i / n_lakers
        pos[node] = np.array([
            center_radius * np.cos(angle),
            center_radius * np.sin(angle)
        ])
    
    # Position other players in outer circle
    other_nodes = [node for node in G.nodes if node not in lakers_players]
    n_others = len(other_nodes)
    for i, node in enumerate(other_nodes):
        angle = 2 * np.pi * i / n_others
        pos[node] = np.array([
            outer_radius * np.cos(angle),
            outer_radius * np.sin(angle)
        ])
    
    return pos

# Visualization
plt.figure(figsize=(40, 40))
pos = custom_layout(G, lakers_players)
edges = G.edges(data=True)
weights = [edge[2]['weight'] for edge in edges]

# Draw the network
nx.draw(G, pos,
        node_color=[node_colors[node] for node in G.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('NBA Players UNIT_MATCHUP_TOV Network\nLakers Players (Gold) in Center', fontsize=16)
plt.show()

# First plot: UNIT_MATCHUP_TOV capability
plt.figure(figsize=(40, 40))

# Create subgraph with only edges to Lakers
edges_to_lakers = [(u, v) for (u, v, d) in G.edges(data=True) 
                   if v in lakers_players]
G_blocks = G.edge_subgraph(edges_to_lakers).copy()

# Remove isolated nodes
G_blocks.remove_nodes_from(list(nx.isolates(G_blocks)))

pos = custom_layout(G_blocks, lakers_players)
weights_to_lakers = [G[u][v]['weight'] for u, v in G_blocks.edges()]
node_colors_filtered = {node: node_colors[node] for node in G_blocks.nodes()}

nx.draw(G_blocks, pos,
        node_color=[node_colors_filtered[node] for node in G_blocks.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights_to_lakers,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('UNIT_MATCHUP_TOV Against Lakers Players', fontsize=16)
plt.show()

# Second plot: Anti-block capability
plt.figure(figsize=(40, 40))

# Create subgraph with only edges from Lakers
edges_from_lakers = [(u, v) for (u, v, d) in G.edges(data=True) 
                     if u in lakers_players]
G_antiblocks = G.edge_subgraph(edges_from_lakers).copy()

# Remove isolated nodes
G_antiblocks.remove_nodes_from(list(nx.isolates(G_antiblocks)))

pos = custom_layout(G_antiblocks, lakers_players)
weights_from_lakers = [G[u][v]['weight'] for u, v in G_antiblocks.edges()]
node_colors_filtered = {node: node_colors[node] for node in G_antiblocks.nodes()}

nx.draw(G_antiblocks, pos,
        node_color=[node_colors_filtered[node] for node in G_antiblocks.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights_from_lakers,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('UNIT_MATCHUP_TOV By Lakers Players', fontsize=16)
plt.show()

## UNIT_MATCHUP_PTS with all connections

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Create a graph
G = nx.DiGraph()

# Initialize node color map
node_colors = {}

# Filter players from Lakers and add nodes
lakers_players = {name: data for name, data in player_data.items() if data['team_name'] == 'Los Angeles Lakers'}
for name in lakers_players:
    G.add_node(name)
    node_colors[name] = 'gold'  # Color for Lakers players

# Add other NBA players as nodes
for name in player_data:
    if name not in lakers_players:
        G.add_node(name)
        node_colors[name] = 'gray'  # Color for other players

# Add edges based on UNIT_MATCHUP_PTS values
for name, data in player_data.items():
    df = data['data_frame']
    if 'OFF_PLAYER_NAME' in df.columns and 'UNIT_MATCHUP_PTS' in df.columns:
        for index, row in df.iterrows():
            if row['UNIT_MATCHUP_PTS'] != 0:
                if row['OFF_PLAYER_NAME'] in G.nodes and name in G.nodes:
                    G.add_edge(row['OFF_PLAYER_NAME'], name, weight=row['UNIT_MATCHUP_PTS'])

# Remove nodes with no edges
nodes_with_no_edges = [node for node in G.nodes if G.degree(node) == 0]
G.remove_nodes_from(nodes_with_no_edges)

# Create custom layout
def custom_layout(G, lakers_players, center_radius=0.3, outer_radius=1.0):
    pos = {}
    
    # Position Lakers players in inner circle
    lakers_nodes = [node for node in G.nodes if node in lakers_players]
    n_lakers = len(lakers_nodes)
    for i, node in enumerate(lakers_nodes):
        angle = 2 * np.pi * i / n_lakers
        pos[node] = np.array([
            center_radius * np.cos(angle),
            center_radius * np.sin(angle)
        ])
    
    # Position other players in outer circle
    other_nodes = [node for node in G.nodes if node not in lakers_players]
    n_others = len(other_nodes)
    for i, node in enumerate(other_nodes):
        angle = 2 * np.pi * i / n_others
        pos[node] = np.array([
            outer_radius * np.cos(angle),
            outer_radius * np.sin(angle)
        ])
    
    return pos

# Visualization
plt.figure(figsize=(40, 40))
pos = custom_layout(G, lakers_players)
edges = G.edges(data=True)
weights = [edge[2]['weight'] for edge in edges]

# Draw the network
nx.draw(G, pos,
        node_color=[node_colors[node] for node in G.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('NBA Players UNIT_MATCHUP_PTS Network\nLakers Players (Gold) in Center', fontsize=16)
plt.show()

# First plot: Block capability
plt.figure(figsize=(40, 40))

# Create subgraph with only edges to Lakers
edges_to_lakers = [(u, v) for (u, v, d) in G.edges(data=True) 
                   if v in lakers_players]
G_blocks = G.edge_subgraph(edges_to_lakers).copy()

# Remove isolated nodes
G_blocks.remove_nodes_from(list(nx.isolates(G_blocks)))

pos = custom_layout(G_blocks, lakers_players)
weights_to_lakers = [G[u][v]['weight'] for u, v in G_blocks.edges()]
node_colors_filtered = {node: node_colors[node] for node in G_blocks.nodes()}

nx.draw(G_blocks, pos,
        node_color=[node_colors_filtered[node] for node in G_blocks.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights_to_lakers,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('UNIT_MATCHUP_PTS Against Lakers Players', fontsize=16)
plt.show()

# Second plot: Anti-block capability
plt.figure(figsize=(40, 40))

# Create subgraph with only edges from Lakers
edges_from_lakers = [(u, v) for (u, v, d) in G.edges(data=True) 
                     if u in lakers_players]
G_antiblocks = G.edge_subgraph(edges_from_lakers).copy()

# Remove isolated nodes
G_antiblocks.remove_nodes_from(list(nx.isolates(G_antiblocks)))

pos = custom_layout(G_antiblocks, lakers_players)
weights_from_lakers = [G[u][v]['weight'] for u, v in G_antiblocks.edges()]
node_colors_filtered = {node: node_colors[node] for node in G_antiblocks.nodes()}

nx.draw(G_antiblocks, pos,
        node_color=[node_colors_filtered[node] for node in G_antiblocks.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights_from_lakers,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('UNIT_MATCHUP_PTS By Lakers Players', fontsize=16)
plt.show()

## Centrality analysis of blockers

In [ ]:
# Analyze blocks against Lakers
G_blocks = G.edge_subgraph([(u, v) for (u, v, d) in G.edges(data=True) if v in lakers_players]).copy()
G_blocks.remove_nodes_from(list(nx.isolates(G_blocks)))

# Calculate weighted in-degree centrality for Lakers (who gets blocked most)
in_degree = dict(G_blocks.in_degree(weight='weight'))
top_blocked = sorted([(node, cent) for node, cent in in_degree.items() if node in lakers_players], 
                    key=lambda x: x[1], reverse=True)

# Calculate weighted out-degree centrality (who blocks Lakers most)
out_degree = dict(G_blocks.out_degree(weight='weight'))
top_blockers = sorted([(node, cent) for node, cent in out_degree.items() if node not in lakers_players], 
                     key=lambda x: x[1], reverse=True)

# Print results
print("Most Blocked Lakers Players:")
for player, blocks in top_blocked[:5]:
    print(f"{player}: {blocks:.2f} blocks")

print("\nTop Blockers Against Lakers:")
for player, blocks in top_blockers[:5]:
    print(f"{player}: {blocks:.2f} blocks")

# Visualize with node sizes based on centrality
plt.figure(figsize=(40, 40))
pos = custom_layout(G_blocks, lakers_players)

# Node sizes based on degree centrality
node_sizes = [out_degree.get(node, 0) * 100 + 700 for node in G_blocks.nodes()]

nx.draw(G_blocks, pos,
        node_color=[node_colors[node] for node in G_blocks.nodes()],
        with_labels=True,
        node_size=node_sizes,
        edge_color=[d['weight'] for (u, v, d) in G_blocks.edges(data=True)],
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('Blocks Against Lakers Players\n(Node size represents blocking frequency)', fontsize=16)
plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Create a graph
G = nx.DiGraph()

# Initialize node color map
node_colors = {}

# Filter players from Lakers and add nodes
lakers_players = {name: data for name, data in player_data.items() if data['team_name'] == 'Los Angeles Lakers'}
for name in lakers_players:
    G.add_node(name)
    node_colors[name] = 'gold'  # Color for Lakers players

# Add other NBA players as nodes
for name in player_data:
    if name not in lakers_players:
        G.add_node(name)
        node_colors[name] = 'gray'  # Color for other players

# Modify edge creation section
for name, data in player_data.items():
    df = data['data_frame']
    if 'OFF_PLAYER_NAME' in df.columns and 'MATCHUP_FG3_PCT' in df.columns and 'MATCHUP_FG3A' in df.columns:
        for index, row in df.iterrows():
            if row['MATCHUP_FG3A'] != 0:  # Only add edge if there were 3-point attempts
                if row['OFF_PLAYER_NAME'] in G.nodes and name in G.nodes:
                    G.add_edge(row['OFF_PLAYER_NAME'], name, weight=row['MATCHUP_FG3_PCT'])

# Remove nodes with no edges
nodes_with_no_edges = [node for node in G.nodes if G.degree(node) == 0]
G.remove_nodes_from(nodes_with_no_edges)

# Create custom layout
def custom_layout(G, lakers_players, center_radius=0.3, outer_radius=1.0):
    pos = {}
    
    # Position Lakers players in inner circle
    lakers_nodes = [node for node in G.nodes if node in lakers_players]
    n_lakers = len(lakers_nodes)
    for i, node in enumerate(lakers_nodes):
        angle = 2 * np.pi * i / n_lakers
        pos[node] = np.array([
            center_radius * np.cos(angle),
            center_radius * np.sin(angle)
        ])
    
    # Position other players in outer circle
    other_nodes = [node for node in G.nodes if node not in lakers_players]
    n_others = len(other_nodes)
    for i, node in enumerate(other_nodes):
        angle = 2 * np.pi * i / n_others
        pos[node] = np.array([
            outer_radius * np.cos(angle),
            outer_radius * np.sin(angle)
        ])
    
    return pos

# Visualization
plt.figure(figsize=(40, 40))
pos = custom_layout(G, lakers_players)
edges = G.edges(data=True)
weights = [edge[2]['weight'] for edge in edges]

# Draw the network
nx.draw(G, pos,
        node_color=[node_colors[node] for node in G.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('NBA Players MATCHUP_FG3_PCT Network\nLakers Players (Gold) in Center', fontsize=16)
plt.show()

# First plot: Block capability
plt.figure(figsize=(40, 40))

# Create subgraph with only edges to Lakers
edges_to_lakers = [(u, v) for (u, v, d) in G.edges(data=True) 
                   if v in lakers_players]
G_blocks = G.edge_subgraph(edges_to_lakers).copy()

# Remove isolated nodes
G_blocks.remove_nodes_from(list(nx.isolates(G_blocks)))

pos = custom_layout(G_blocks, lakers_players)
weights_to_lakers = [G[u][v]['weight'] for u, v in G_blocks.edges()]
node_colors_filtered = {node: node_colors[node] for node in G_blocks.nodes()}

nx.draw(G_blocks, pos,
        node_color=[node_colors_filtered[node] for node in G_blocks.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights_to_lakers,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('FG3_PCT Against Lakers Players', fontsize=16)
plt.show()

# Second plot: Anti-block capability
plt.figure(figsize=(40, 40))

# Create subgraph with only edges from Lakers
edges_from_lakers = [(u, v) for (u, v, d) in G.edges(data=True) 
                     if u in lakers_players]
G_antiblocks = G.edge_subgraph(edges_from_lakers).copy()

# Remove isolated nodes
G_antiblocks.remove_nodes_from(list(nx.isolates(G_antiblocks)))

pos = custom_layout(G_antiblocks, lakers_players)
weights_from_lakers = [G[u][v]['weight'] for u, v in G_antiblocks.edges()]
node_colors_filtered = {node: node_colors[node] for node in G_antiblocks.nodes()}

nx.draw(G_antiblocks, pos,
        node_color=[node_colors_filtered[node] for node in G_antiblocks.nodes()],
        with_labels=True,
        node_size=700,
        edge_color=weights_from_lakers,
        width=2.0,
        edge_cmap=plt.cm.Blues,
        font_size=8,
        font_weight='bold')

plt.title('FG3_PCT By Lakers Players', fontsize=16)
plt.show()